In [40]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
import numpy as np
import scipy.stats as stats
from matplotlib import pyplot
import seaborn as sns

In [2]:
url = "data/OrinaAntLimpio.csv"
datos = pd.read_csv(url, encoding="unicode_escape", index_col=[0])
datos.head()

,numVol,Endulzante,CA.Gluc,DHPAA.Gluc,TFA.Gluc,TFA.Sulfate,Ácido.Vanílico..VA.,Tiempo,Sexo,Peso,IMC,Grasa,IRCV,Bpmin,Bpmax,Frec
1,2,ST,3.011927,4.436169,0.141377,144.904736,79.439633,0,MUJER,70.2,27.4,38.6,8,81,107,69
2,3,ST,0.080472,2.387064,0.378852,70.095759,11.609222,0,MUJER,80.4,26.9,39.2,11,75,107,64
3,8,ST,0.074026,1.582581,2.209738,73.493403,9.064309,0,MUJER,66.8,25.1,38.5,6,76,107,79
4,9,ST,0.034608,1.762405,0.161311,15.797066,10.368426,0,MUJER,64.7,28.0,44.7,9,76,107,74
5,10,ST,0.046809,0.199703,0.075798,10.145476,8.639885,0,MUJER,91.3,32.7,44.2,10,88,150,80


In [7]:
#datos_num = datos.select_dtypes(include=np.number).drop("numVol",1)

# create a scaler object

scaler = MinMaxScaler()

#fit & transform the data

#datos_norm = pd.DataFrame(scaler.fit_transform(datos_num), columns = datos_num.columns)
#datos_norm = datos.apply(lambda x: scaler.fit_transform(x) if x[.name] not in ["numVol", "Tiempo", "Endulzante", "Sexo"] else x)
numCols = datos.select_dtypes(include=np.number).drop("numVol",1).columns
datos[numCols] = scaler.fit_transform(datos[numCols])


C:\Users\CRISTI~1\AppData\Local\Temp/ipykernel_20568/4169978868.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  numCols = datos.select_dtypes(include=np.number).drop("numVol",1).columns


In [41]:
z_scores = stats.zscore(datos[numCols])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
datosClean = datos[filtered_entries]
datosNum = datosClean[numCols]
datosNum.columns

Index(['CA.Gluc', 'DHPAA.Gluc', 'TFA.Gluc', 'TFA.Sulfate',
       'Ácido.Vanílico..VA.', 'Peso', 'IMC', 'Grasa', 'IRCV', 'Bpmin', 'Bpmax',
       'Frec'],
      dtype='object')

In [45]:
## affinity propagation clustering
from sklearn.cluster import AffinityPropagation

model = AffinityPropagation()
model.fit(datosNum)

yhat = model.predict(datosNum)
clusters = np.unique(yhat)
datosClust = pd.concat((datosClean, pd.Series(yhat)), axis = 1, ignore_index=True)
datosClust.head()
#sns.lmplot(x='Grasa',y='Peso',data=datosClust,hue='yhat',fit_reg=False)



C:\ProgramData\Anaconda3\envs\deepLcurse\lib\site-packages\sklearn\cluster\_affinity_propagation.py:148: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,3.0,ST,0.020402,0.154252,0.011191,0.075332,0.019029,0,MUJER,0.436893,0.353383,0.710602,0.466667,0.238806,0.345238,0.372881,9.0
3,8.0,ST,0.018567,0.101799,0.073855,0.079011,0.014788,0,MUJER,0.172816,0.218045,0.690544,0.133333,0.253731,0.345238,0.627119,4.0
4,9.0,ST,0.007350,0.113523,0.003745,0.016538,0.016962,0,MUJER,0.132039,0.436090,0.868195,0.333333,0.253731,0.345238,0.542373,13.0
